In [ ]:
import pandas as pd

df = pd.read_csv('/content/demand_forecasting_data.csv')
df.head()


,Date,Product_ID,Base_Sales,Marketing_Campaign,Marketing_Effect,Seasonal_Trend,Seasonal_Effect,Price,Discount,Competitor_Price,Stock_Availability,Public_Holiday,Demand
0,2019-01-01,P002,65,Social Media,1.634270,Spring,1.0,73.496059,0.078198,64.173418,491,False,60570
1,2019-01-01,P004,94,Social Media,1.240566,Summer,1.2,74.271862,0.182151,69.571391,135,True,18143
2,2019-01-01,P003,125,Radio,1.087600,Summer,1.2,35.274616,0.102592,27.331268,180,False,37412
3,2019-01-01,P004,128,TV,1.831657,Winter,0.8,79.524248,0.196125,73.429502,227,False,40773
4,2019-01-01,P001,51,Radio,1.285161,Fall,1.1,96.237402,0.079253,88.243871,338,False,26917


In [ ]:
print(df.columns.tolist())

['Date', 'Product_ID', 'Base_Sales', 'Marketing_Campaign', 'Marketing_Effect', 'Seasonal_Trend', 'Seasonal_Effect', 'Price', 'Discount', 'Competitor_Price', 'Stock_Availability', 'Public_Holiday', 'Demand']


In [ ]:
cols = [
    'Base_Sales',
    'Marketing_Effect',
    'Seasonal_Effect',
    'Price',
    'Discount',
    'Competitor_Price',
    'Stock_Availability',
    'Public_Holiday',
    'Demand'
]

df_model = df[cols].dropna()
df_model.head()


,Base_Sales,Marketing_Effect,Seasonal_Effect,Price,Discount,Competitor_Price,Stock_Availability,Public_Holiday,Demand
0,65,1.634270,1.0,73.496059,0.078198,64.173418,491,False,60570
1,94,1.240566,1.2,74.271862,0.182151,69.571391,135,True,18143
2,125,1.087600,1.2,35.274616,0.102592,27.331268,180,False,37412
3,128,1.831657,0.8,79.524248,0.196125,73.429502,227,False,40773
4,51,1.285161,1.1,96.237402,0.079253,88.243871,338,False,26917


In [ ]:
df[['Marketing_Campaign', 'Seasonal_Trend']].head(10)

,Marketing_Campaign,Seasonal_Trend
0,Social Media,Spring
1,Social Media,Summer
2,Radio,Summer
3,TV,Winter
4,Radio,Fall
5,NaN,Spring
6,Radio,Spring
7,Social Media,Summer
8,NaN,Fall
9,NaN,Fall


In [ ]:
df[['Marketing_Campaign', 'Seasonal_Trend']].isna().sum()

,0
Marketing_Campaign,7067
Seasonal_Trend,0


In [ ]:
df[['Marketing_Effect', 'Seasonal_Effect']].describe()

,Marketing_Effect,Seasonal_Effect
count,35000.000000,35000.000000
mean,1.399607,1.023937
std,0.377013,0.148251
min,0.960101,0.800000
25%,1.057890,0.800000
50%,1.320028,1.000000
75%,1.628659,1.100000
max,2.399656,1.200000


In [ ]:
import pandas as pd

num_cols = [
    'Base_Sales',
    'Marketing_Effect',
    'Seasonal_Effect',
    'Price',
    'Discount',
    'Competitor_Price',
    'Stock_Availability',
    'Public_Holiday',
    'Demand'
]

df_model = df[num_cols].copy()

In [ ]:
df_cat = df[['Marketing_Campaign', 'Seasonal_Trend']].fillna('Unknown')

In [ ]:
df_cat_encoded = pd.get_dummies(df_cat, drop_first=True)

In [ ]:
df_model = pd.concat([df_model, df_cat_encoded], axis=1)
df_model = df_model.dropna()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

df_model['demand_label'] = pd.qcut(df_model['Demand'], q=3, labels=['Low','Medium','High'])

X = df_model.drop(['Demand','demand_label'], axis=1)
y = df_model['demand_label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

clf = RandomForestClassifier(n_estimators=200, random_state=42)
clf.fit(X_train, y_train)

print("Demand model accuracy:", clf.score(X_test, y_test))

Demand model accuracy: 0.938


In [ ]:
from sklearn.ensemble import RandomForestRegressor

X_price = df_model.drop(['Price','Demand','demand_label'], axis=1)
y_price = df_model['Price']

X_train_p, X_test_p, y_train_p, y_test_p = train_test_split(X_price, y_price, test_size=0.2, random_state=42)

reg = RandomForestRegressor(n_estimators=200, random_state=42)
reg.fit(X_train_p, y_train_p)

print("Pricing model R2:", reg.score(X_test_p, y_test_p))

Pricing model R2: 0.9827646859669665


In [ ]:
# Define stock action rules
def stock_action(row):
    if row['demand_label'] == 'High' and row['Stock_Availability'] < 100:
        return 'Restock'
    elif row['demand_label'] == 'Low' and row['Stock_Availability'] > 300:
        return 'Reduce'
    else:
        return 'Hold'

df_model['stock_action'] = df_model.apply(stock_action, axis=1)
df_model['stock_action'].value_counts()

,count
stock_action,
Hold,34486
Reduce,514


In [ ]:
from sklearn.ensemble import RandomForestClassifier

X_stock = df_model.drop(['Demand', 'demand_label', 'stock_action', 'Price'], axis=1)
y_stock = df_model['stock_action']

X_train_s, X_test_s, y_train_s, y_test_s = train_test_split(X_stock, y_stock, test_size=0.2, random_state=42)

stock_clf = RandomForestClassifier(n_estimators=200, random_state=42)
stock_clf.fit(X_train_s, y_train_s)

print("Stock action model accuracy:", stock_clf.score(X_test_s, y_test_s))

Stock action model accuracy: 0.9928571428571429


In [ ]:
# Save feature lists right after training
price_features = X_price.columns.tolist()
demand_features = X.columns.tolist()
stock_features  = X_stock.columns.tolist()

In [ ]:
idx = X_test.index[0]
sample_row = df_model.loc[[idx]]

sample_price  = sample_row[price_features]
sample_demand = sample_row[demand_features]
sample_stock  = sample_row[stock_features]

ai_price  = reg.predict(sample_price)[0]
ai_demand = clf.predict(sample_demand)[0]
ai_stock  = stock_clf.predict(sample_stock)[0]

print("AI Recommended Price:", round(ai_price, 2))
print("Predicted Demand:", ai_demand)
print("Stock Action:", ai_stock)

AI Recommended Price: 45.41
Predicted Demand: Low
Stock Action: Hold


In [ ]:
import joblib

joblib.dump(reg, 'price_model.pkl')
joblib.dump(clf, 'demand_model.pkl')
joblib.dump(stock_clf, 'stock_model.pkl')

joblib.dump(price_features, 'price_features.pkl')
joblib.dump(demand_features, 'demand_features.pkl')
joblib.dump(stock_features, 'stock_features.pkl')

['stock_features.pkl']